<a href="https://colab.research.google.com/github/REICHIYAN/REI/blob/master/UT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd

/content


In [ ]:
!mkdir -p chapter7
!echo "file" > chapter7/LiquidityProvider.py

In [ ]:
!ls

chapter7  sample_data


In [ ]:
import unittest
from chapter7.LiquidityProvider import LiquidityProvider


class TestMarketSimulator(unittest.TestCase):
    def setUp(self):
        self.liquidity_provider = LiquidityProvider()

    def test_add_liquidity(self):
        self.liquidity_provider.generate_random_order()
        self.assertEqual(self.liquidity_provider.orders[0]['id'],0)
        self.assertEqual(self.liquidity_provider.orders[0]['side'], 'buy')
        self.assertEqual(self.liquidity_provider.orders[0]['quantity'], 700)
        self.assertEqual(self.liquidity_provider.orders[0]['price'], 11)

In [ ]:
import unittest
from chapter7.MarketSimulator import MarketSimulator


class TestMarketSimulator(unittest.TestCase):

    def setUp(self):
        self.market_simulator = MarketSimulator()

    def test_accept_order(self):
        self.market_simulator
        order1 = {
            'id': 10,
            'price': 219,
            'quantity': 10,
            'side': 'bid',
            'action' : 'New'
        }
        self.market_simulator.handle_order(order1)
        self.assertEqual(len(self.market_simulator.orders),1)
        self.assertEqual(self.market_simulator.orders[0]['status'], 'accepted')

    def test_accept_order(self):
        self.market_simulator
        order1 = {
            'id': 10,
            'price': 219,
            'quantity': 10,
            'side': 'bid',
            'action' : 'Amend'
        }
        self.market_simulator.handle_order(order1)
        self.assertEqual(len(self.market_simulator.orders),0)

In [ ]:
import unittest
from chapter7.OrderBook import OrderBook


class TestOrderBook(unittest.TestCase):

    def setUp(self):
        self.reforderbook = OrderBook()

    def test_handlenew(self):
        order1 = {
            'id': 1,
            'price': 219,
            'quantity': 10,
            'side': 'bid',
            'action': 'new'
        }

        ob_for_aapl = self.reforderbook
        ob_for_aapl.handle_order(order1)
        order2 = order1.copy()
        order2['id'] = 2
        order2['price'] = 220
        ob_for_aapl.handle_order(order2)
        order3 = order1.copy()
        order3['price'] = 223
        order3['id'] = 3
        ob_for_aapl.handle_order(order3)
        order4 = order1.copy()
        order4['side'] = 'ask'
        order4['price'] = 220
        order4['id'] = 4
        ob_for_aapl.handle_order(order4)
        order5 = order4.copy()
        order5['price'] = 223
        order5['id'] = 5
        ob_for_aapl.handle_order(order5)
        order6 = order4.copy()
        order6['price'] = 221
        order6['id'] = 6
        ob_for_aapl.handle_order(order6)

        self.assertEqual(ob_for_aapl.list_bids[0]['id'],3)
        self.assertEqual(ob_for_aapl.list_bids[1]['id'], 2)
        self.assertEqual(ob_for_aapl.list_bids[2]['id'], 1)
        self.assertEqual(ob_for_aapl.list_asks[0]['id'],4)
        self.assertEqual(ob_for_aapl.list_asks[1]['id'], 6)
        self.assertEqual(ob_for_aapl.list_asks[2]['id'], 5)


    def test_handleamend(self):
        self.test_handlenew()
        order1 = {
            'id': 1,
            'quantity': 5,
            'action': 'modify'
        }
        self.reforderbook.handle_order(order1)

        self.assertEqual(self.reforderbook.list_bids[2]['id'], 1)
        self.assertEqual(self.reforderbook.list_bids[2]['quantity'], 5)


    def test_handledelete(self):
        self.test_handlenew()
        order1 = {
            'id': 1,
            'action': 'delete'
        }
        self.assertEqual(len(self.reforderbook.list_bids), 3)
        self.reforderbook.handle_order(order1)
        self.assertEqual(len(self.reforderbook.list_bids), 2)

    def test_generate_book_event(self):
        order1 = {
            'id': 1,
            'price': 219,
            'quantity': 10,
            'side': 'bid',
            'action': 'new'
        }

        ob_for_aapl = self.reforderbook
        self.assertEqual(ob_for_aapl.handle_order(order1),
                         {'bid_price': 219, 'bid_quantity': 10,
                          'offer_price': -1, 'offer_quantity': -1})
        order2 = order1.copy()
        order2['id'] = 2
        order2['price'] = 220
        order2['side'] = 'ask'
        self.assertEqual(ob_for_aapl.handle_order(order2),
        {'bid_price': 219, 'bid_quantity': 10,
         'offer_price': 220, 'offer_quantity': 10})



#if __name__ == '__main__':
#    unittest.main()

In [ ]:
import unittest
from chapter7.OrderManager import OrderManager


class TestOrderBook(unittest.TestCase):

    def setUp(self):
        self.order_manager = OrderManager()

    def test_receive_order_from_trading_strategy(self):
        order1 = {
            'id': 10,
            'price': 219,
            'quantity': 10,
            'side': 'bid',
        }
        self.order_manager.handle_order_from_trading_strategy(order1)
        self.assertEqual(len(self.order_manager.orders),1)
        self.order_manager.handle_order_from_trading_strategy(order1)
        self.assertEqual(len(self.order_manager.orders),2)
        self.assertEqual(self.order_manager.orders[0]['id'],1)
        self.assertEqual(self.order_manager.orders[1]['id'],2)

    def test_receive_order_from_trading_strategy_error(self):
        order1 = {
            'id': 10,
            'price': -219,
            'quantity': 10,
            'side': 'bid',
        }
        self.order_manager.handle_order_from_trading_strategy(order1)
        self.assertEqual(len(self.order_manager.orders),0)

    def display_orders(self):
        for o in self.order_manager.orders:
            print(o)

    def test_receive_from_gateway_filled(self):
        self.test_receive_order_from_trading_strategy()
        orderexecution1 = {
            'id': 2,
            'price': 13,
            'quantity': 10,
            'side': 'bid',
            'status' : 'filled'
        }
        # self.display_orders()
        self.order_manager.handle_order_from_gateway(orderexecution1)
        self.assertEqual(len(self.order_manager.orders), 1)

    def test_receive_from_gateway_acked(self):
        self.test_receive_order_from_trading_strategy()
        orderexecution1 = {
            'id': 2,
            'price': 13,
            'quantity': 10,
            'side': 'bid',
            'status' : 'acked'
        }
        # self.display_orders()
        self.order_manager.handle_order_from_gateway(orderexecution1)
        self.assertEqual(len(self.order_manager.orders), 2)
        self.assertEqual(self.order_manager.orders[1]['status'], 'acked')

In [ ]:
import unittest
from chapter7.LiquidityProvider import LiquidityProvider
from chapter7.TradingStrategy import TradingStrategy
from chapter7.MarketSimulator import MarketSimulator
from chapter7.OrderManager import OrderManager
from chapter7.OrderBook import OrderBook
from collections import deque

class TestTradingSimulation(unittest.TestCase):
    def setUp(self):
        self.lp_2_gateway=deque()
        self.ob_2_ts = deque()
        self.ts_2_om = deque()
        self.ms_2_om = deque()
        self.om_2_ts = deque()
        self.gw_2_om = deque()
        self.om_2_gw = deque()




        self.lp=LiquidityProvider(self.lp_2_gateway)
        self.ob=OrderBook(self.lp_2_gateway, self.ob_2_ts)
        self.ts=TradingStrategy(self.ob_2_ts,self.ts_2_om,self.om_2_ts)
        self.ms=MarketSimulator(self.om_2_gw,self.gw_2_om)
        self.om=OrderManager(self.ts_2_om, self.om_2_ts,self.om_2_gw,self.gw_2_om)



    def test_add_liquidity(self):
        # Order sent from the exchange to the trading system
        order1 = {
            'id': 1,
            'price': 219,
            'quantity': 10,
            'side': 'bid',
            'action': 'new'
        }
        self.lp.insert_manual_order(order1)
        self.assertEqual(len(self.lp_2_gateway),1)
        self.ob.handle_order_from_gateway()
        self.assertEqual(len(self.ob_2_ts), 1)
        self.ts.handle_input_from_bb()
        self.assertEqual(len(self.ts_2_om), 0)
        order2 = {
            'id': 2,
            'price': 218,
            'quantity': 10,
            'side': 'ask',
            'action': 'new'
        }
        self.lp.insert_manual_order(order2.copy())
        self.assertEqual(len(self.lp_2_gateway),1)
        self.ob.handle_order_from_gateway()
        self.assertEqual(len(self.ob_2_ts), 1)
        self.ts.handle_input_from_bb()
        self.assertEqual(len(self.ts_2_om), 2)
        self.om.handle_input_from_ts()
        self.assertEqual(len(self.ts_2_om), 1)
        self.assertEqual(len(self.om_2_gw), 1)
        self.om.handle_input_from_ts()
        self.assertEqual(len(self.ts_2_om), 0)
        self.assertEqual(len(self.om_2_gw), 2)
        self.ms.handle_order_from_gw()
        self.assertEqual(len(self.gw_2_om), 1)
        self.ms.handle_order_from_gw()
        self.assertEqual(len(self.gw_2_om), 2)
        self.om.handle_input_from_market()
        self.om.handle_input_from_market()
        self.assertEqual(len(self.om_2_ts), 2)
        self.ts.handle_response_from_om()
        self.assertEqual(self.ts.get_pnl(),0)
        self.ms.fill_all_orders()
        self.assertEqual(len(self.gw_2_om), 2)
        self.om.handle_input_from_market()
        self.om.handle_input_from_market()
        self.assertEqual(len(self.om_2_ts), 3)
        self.ts.handle_response_from_om()
        self.assertEqual(len(self.om_2_ts), 2)
        self.ts.handle_response_from_om()
        self.assertEqual(len(self.om_2_ts), 1)
        self.ts.handle_response_from_om()
        self.assertEqual(len(self.om_2_ts), 0)
        self.assertEqual(self.ts.get_pnl(),10)

In [ ]:
import unittest
from chapter7.TradingStrategy import TradingStrategy


class TestMarketSimulator(unittest.TestCase):

    def setUp(self):
        self.trading_strategy= TradingStrategy()


    def test_receive_top_of_book(self):
        book_event = {
            "bid_price" : 12,
            "bid_quantity" : 100,
            "offer_price" : 11,
            "offer_quantity" : 150
        }
        self.trading_strategy.handle_book_event(book_event)
        self.assertEqual(len(self.trading_strategy.orders), 2)
        self.assertEqual(self.trading_strategy.orders[0]['side'], 'sell')
        self.assertEqual(self.trading_strategy.orders[1]['side'], 'buy')
        self.assertEqual(self.trading_strategy.orders[0]['price'], 12)
        self.assertEqual(self.trading_strategy.orders[1]['price'], 11)
        self.assertEqual(self.trading_strategy.orders[0]['quantity'], 100)
        self.assertEqual(self.trading_strategy.orders[1]['quantity'], 100)
        self.assertEqual(self.trading_strategy.orders[0]['action'], 'no_action')
        self.assertEqual(self.trading_strategy.orders[1]['action'], 'no_action')


    def test_rejected_order(self):
        self.test_receive_top_of_book()
        order_execution = {
            'id': 1,
            'price': 12,
            'quantity': 100,
            'side': 'sell',
            'status' : 'rejected'
        }
        self.trading_strategy.handle_market_response(order_execution)
        self.assertEqual(self.trading_strategy.orders[0]['side'], 'buy')
        self.assertEqual(self.trading_strategy.orders[0]['price'], 11)
        self.assertEqual(self.trading_strategy.orders[0]['quantity'], 100)
        self.assertEqual(self.trading_strategy.orders[0]['status'], 'new')


    def test_filled_order(self):
        self.test_receive_top_of_book()
        order_execution = {
            'id': 1,
            'price': 11,
            'quantity': 100,
            'side': 'sell',
            'status' : 'filled'
        }
        self.trading_strategy.handle_market_response(order_execution)
        self.assertEqual(len(self.trading_strategy.orders),1)

        order_execution = {
            'id': 2,
            'price': 12,
            'quantity': 100,
            'side': 'buy',
            'status' : 'filled'
        }
        self.trading_strategy.handle_market_response(order_execution)
        self.assertEqual(self.trading_strategy.position, 0)
        self.assertEqual(self.trading_strategy.cash, 10100)
        self.assertEqual(self.trading_strategy.pnl, 100)